# Imports

In [ ]:
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
# sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [ ]:
dataset_name = 'ucberkeley'
model_name = 'bert-base-uncased'
run_no = 1
# change to f'../../results/{dataset_name}' when using inside one of the data subfolders
result_folder = f'../results/{dataset_name}/run {run_no}'
test_csv_filepath = os.path.join(result_folder, 'test.csv')

model_folder = f'{result_folder}/{model_name}' # for this particular model
result_folder = os.path.join(model_folder, 'normal')
result_filepath = os.path.join(result_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [ ]:
result = pd.read_csv(result_filepath)
test_df = pd.read_csv(test_csv_filepath)
test_df.head(2)

## Split test result

In [ ]:
# only calculate test result
result = result[result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)

## Prepare test data

In [ ]:
# result has id column which is the same as the text ids from raw dataset
raw_id_column = 'comment_id'
test_df.rename({'comment_id': id_column}, axis=1, inplace=True)

## Merge

In [ ]:
result = result.merge(test_df.drop(columns=['labels']), on=id_column, how='inner').reset_index(drop=True)
result.head()

# Evaluation

## Convert probability to prediction

In [ ]:
result[prediction_column] = result[probability_column] >=0.5

## Identity groups

In [ ]:
group_map = {
    'gender': {
        'unprivileged':['target_gender_women'],
        'privileged':['target_gender_men']
    },
    'race': {
        'unprivileged':['target_race_black'],
        'privileged': ['target_race_white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

### Binarize identity and target columns

In [ ]:
result = binarize(result, [target_column] + identities)

## Bias

In [ ]:
bias_results = get_all_bias(group_map, result)
bias_results

In [ ]:
bias_results.to_csv(os.path.join(model_folder, 'bias.csv'), index=False)

## Overall metrics

In [ ]:
overall_results = get_overall_results(group_map, result)
overall_results

In [ ]:
overall_results.to_csv(os.path.join(model_folder, 'overall_results.csv'), index=False)